In [2]:
# !pip install dotenv fyers_api
from fyers_api import fyersModel
from fyers_api import accessToken
import pandas as pd
import datetime as dt
import os
from dotenv import load_dotenv
load_dotenv("../.env")
# print("running")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 7


True

In [3]:
client_id=os.environ.get('API_CLIENT_ID')
secret_key=os.environ.get('API_SECRET_KEY')
redirect_uri=os.environ.get('API_REDIRECT_URL')


In [4]:
session=accessToken.SessionModel(client_id=client_id,
secret_key=secret_key,redirect_uri=redirect_uri, 
response_type='code', grant_type='authorization_code')

In [5]:
auth_code_provider = session.generate_authcode()
print(auth_code_provider)

https://api.fyers.in/api/v2/generate-authcode?client_id=QW4JS7GAG2-100&redirect_uri=https%3A%2F%2F127.0.0.1%3A5000%2Flogin&response_type=code&state=None


In [6]:
auth_code=str(input("Enter the auth_code: "))

In [7]:
session.set_token(auth_code)
response = session.generate_token()
access_token = response["access_token"]
if(access_token is None):
    print("Access token is not generated")
else:
    print("Access token is generated")
    

Access token is generated


In [8]:
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token)
is_async = True

In [9]:
fyers.get_profile()

ERR: logEntryFunc: get_profile : [Errno 13] Permission denied: '/2022-08-04.txt'


{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'XA15265',
  'name': 'AIYAZ ANWAR QURESHI',
  'image': None,
  'display_name': None,
  'pin_change_date': '31-07-2022 00:55:39',
  'email_id': 'qureshi.aiyaz123@gmail.com',
  'pwd_change_date': '31-07-2022 00:54:23',
  'PAN': '---------',
  'pwd_to_expire': 146}}

In [13]:
# Getting Data
def dynamicDataFetching(startDate,endDate,symbol):
    # Creating a dataframe to store the data
    df = pd.DataFrame(columns=["date","open","high","low","close","volume"])
    today = dt.date.today()
    range_from=startDate
    while(range_from<=today):
    # Check whether the upcoming endDate is lesser than today if we add 99D or else make it today
        if((range_from + dt.timedelta(days=99))<today):
            range_to = range_from + dt.timedelta(days=99)
        else:
            range_to = today

        # Api call 
        data = {"symbol":symbol,"resolution":"5","date_format":"1","range_from":range_from,"range_to":range_to,"cont_flag":"1"}
        res = fyers.history(data)
        res = res["candles"]
        
        # Converting the data into pandas dataframe 
        tempDf = pd.DataFrame(columns=["date","open","high","low","close","volume"])
        for i in range(len(res)):
             tempDf.loc[i]= [
                res[i][0],
                res[i][1],
                res[i][2],
                res[i][3],
                res[i][4],
                res[i][5]]
       
        # Appending the data to the main dataframe
        df = pd.concat([df,tempDf])

        # Updating the range_from to last date of last api call
        range_from=range_to+dt.timedelta(days=1)
    return df

In [14]:
startDate = dt.date(2021,7,1)
endDate = dt.date.today()
symbol="NSE:SBIN-EQ"

df = dynamicDataFetching(startDate,endDate,symbol)
print(df.dtypes)
print(df.describe())

ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-04.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-04.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-04.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-04.txt'
date      object
open      object
high      object
low       object
close     object
volume    object
dtype: object
                date     open     high      low    close   volume
count   2.048400e+04  20484.0  20484.0  20484.0  20484.0  20484.0
unique  2.048400e+04   2696.0   2681.0   2682.0   2705.0  19942.0
top     1.625111e+09    490.0    516.0    430.0    515.0  65077.0
freq    1.000000e+00     29.0     46.0     39.0     33.0      4.0


In [15]:
print(df.head())

           date    open    high     low   close     volume
0  1625111100.0   420.3  420.75  417.15   419.0  1076303.0
1  1625111400.0   419.0   419.0  417.25   418.0   708973.0
2  1625111700.0   418.0  418.15   417.4   417.7   291299.0
3  1625112000.0   417.5   418.4   417.3   418.1   416255.0
4  1625112300.0  418.15   419.4  417.95  419.05   425712.0


In [16]:
# Converting everything to playable format + changing the time zone from utc to ist
df["date"]= pd.to_datetime(df["date"],unit='s',utc=True).dt.tz_convert('Asia/Kolkata')
df.set_index("date",inplace=True)
df["open"]=df["open"].astype(float)
df["high"]=df["high"].astype(float)
df["low"]=df["low"].astype(float)
df["close"]=df["close"].astype(float)
df["volume"]=df["volume"].astype(float)

print(df.head())
print("\nData types :\n" , df.dtypes)

                             open    high     low   close     volume
date                                                                
2021-07-01 09:15:00+05:30  420.30  420.75  417.15  419.00  1076303.0
2021-07-01 09:20:00+05:30  419.00  419.00  417.25  418.00   708973.0
2021-07-01 09:25:00+05:30  418.00  418.15  417.40  417.70   291299.0
2021-07-01 09:30:00+05:30  417.50  418.40  417.30  418.10   416255.0
2021-07-01 09:35:00+05:30  418.15  419.40  417.95  419.05   425712.0

Data types :
 open      float64
high      float64
low       float64
close     float64
volume    float64
dtype: object


In [18]:
# Export df to csv
fileName = "SBI.csv"
# df.to_csv(f"../data/{fileName}")